In [23]:
import sys
sys.path.append('..')

import os
import os.path as osp
import time
import pickle
import numpy as np
import pandas as pd
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, TerminateOnNaN, TensorBoard
from sklearn.model_selection import train_test_split

# custom modules
from libs.utilities import load_dataset, create_autoencoder_model, load_autoencoder_model, load_autoencoder_lossfunc
from libs.model_utils import LossLayer
from libs.data_generator import DataGenerator
from libs.processing import pink_noise, s_to_power

os.getcwd()

In [24]:
#### PARAMETERS ####

model_source = '../models/fwe2014.json'


'/home/christie/SingleChannelDenoising_source/models'

In [25]:
#### OTHER VARS ####
# loss function: data slice under consideration
time_slice = slice(None)
initial_epoch = 0
input_shape = (256, 16, 1)


In [45]:
### SHOW MODEL FILE ###
import json
import pprint 

pprint.pprint(json.load(open(model_source)))

{'_meta': {'descr': 'lstm-drnn',
           'source': 'F. Weninger et al, Discriminatively Trained Recurrent '
                     'Neural Networks for Single-Channel Speech Separation, '
                     '2014'},
 'architecture': [{'layer_args': {'dims': [2, 1, 3]}, 'layer_type': 'permute'},
                  {'layer_args': {'target_shape': [16, -1]},
                   'layer_type': 'reshape'},
                  {'layer_args': {'return_sequences': True, 'units': 256},
                   'layer_type': 'lstm'},
                  {'layer_args': {'return_sequences': True, 'units': 32},
                   'layer_type': 'lstm'},
                  {'layer_args': {'return_sequences': True, 'units': 32},
                   'layer_type': 'lstm'},
                  {'layer_args': {'return_sequences': True, 'units': 256},
                   'layer_type': 'lstm'},
                  {'layer_args': {'target_shape': [16, -1, 1]},
                   'layer_type': 'reshape'},
                  {'

In [46]:
### TRY AND GENERATE MODEL ###
# NOTE: restart the kernel and rerun everything after editing an imported script or lib
model, lossfunc = create_autoencoder_model(
    model_source, input_shape, time_slice=time_slice)

[u] Creating autoencoder model from ../models/fwe2014.json
[u] Model factory parameters: {'model_architecture': {'architecture': [{'layer_type': 'permute', 'layer_args': {'dims': [2, 1, 3]}}, {'layer_type': 'reshape', 'layer_args': {'target_shape': [16, -1]}}, {'layer_type': 'lstm', 'layer_args': {'units': 256, 'return_sequences': True}}, {'layer_type': 'lstm', 'layer_args': {'units': 32, 'return_sequences': True}}, {'layer_type': 'lstm', 'layer_args': {'units': 32, 'return_sequences': True}}, {'layer_type': 'lstm', 'layer_args': {'units': 256, 'return_sequences': True}}, {'layer_type': 'reshape', 'layer_args': {'target_shape': [16, -1, 1]}}, {'layer_type': 'permute', 'layer_args': {'dims': [2, 1, 3]}}], '_meta': {'source': 'F. Weninger et al, Discriminatively Trained Recurrent Neural Networks for Single-Channel Speech Separation, 2014', 'descr': 'lstm-drnn'}}, 'input_shape': (256, 16, 1), 'time_slice': slice(None, None, None)}
Tensor("input_17:0", shape=(?, 256, 16, 1), dtype=float32)

In [47]:
### VERIFY GENERATED MODEL ###
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 256, 16, 1)        0         
_________________________________________________________________
permute_18 (Permute)         (None, 16, 256, 1)        0         
_________________________________________________________________
reshape_7 (Reshape)          (None, 16, 256)           0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 16, 256)           525312    
_________________________________________________________________
lstm_24 (LSTM)               (None, 16, 32)            36992     
_________________________________________________________________
lstm_25 (LSTM)               (None, 16, 32)            8320      
_________________________________________________________________
lstm_26 (LSTM)               (None, 16, 256)           295936    
__________

_____
# Can we use the built-in Keras Model exporters (dict, yaml, json)?

In [18]:
### TEST KERAS BUILTIN JSON IMPORT/EXPORT
from models.model_example import AEModelFactory
obj = AEModelFactory(input_shape, 3, 16)
model2 = obj.get_model()

In [19]:
model2.get_layer('encoder').summary()
model2.get_layer('decoder').summary()
model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 256, 16, 1)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 8, 4)         40        
_________________________________________________________________
batch_normalization_23 (Batc (None, 128, 8, 4)         16        
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 4, 8)          296       
_________________________________________________________________
batch_normalization_24 (Batc (None, 64, 4, 8)          32        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 2, 16)         1168      
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 2, 16)         64        
__________

In [17]:
from pprint import pprint
model_config = model2.get_config()
print(model_config)
#pprint(model_config)  ### looks prettier but takes all the space

{'name': 'model_6', 'layers': [{'name': 'input', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 256, 16, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input'}, 'inbound_nodes': []}, {'name': 'encoder', 'class_name': 'Model', 'config': {'name': 'encoder', 'layers': [{'name': 'input_4', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 256, 16, 1), 'dtype': 'float32', 'sparse': False, 'name': 'input_4'}, 'inbound_nodes': []}, {'name': 'conv2d_5', 'class_name': 'Conv2D', 'config': {'name': 'conv2d_5', 'trainable': True, 'filters': 4, 'kernel_size': (3, 3), 'strides': (2, 2), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activit

In [20]:
# well i guess that's a no